In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import hashlib
import plotly
import plotly.express as px



In [ ]:

df = pd.read_csv(r"athlete_events.csv")


In [ ]:
df = df.fillna({
    'Age': df['Age'].median(),
    'Height': df['Height'].median(),
    'Weight': df['Weight'].median()
})

df = df.astype({'Age': 'uint8', 'Height': 'uint8', 'Weight': 'uint8', 'ID': 'uint32' , 'Year': 'int16'})
df.info()

# ITALIA

In [ ]:
# Anonymisera namn med SHA-256
df["Name"] = df["Name"].apply(
    lambda x: hashlib.sha256(str(x).encode()).hexdigest()
)

# Fyll tomma medaljer med "None" = ingen medalj
df["Medal"] = df["Medal"].fillna("None")


ita = df[df["NOC"] == "ITA"].copy()

ita.head()


In [ ]:
# Top 10 sporter där Italien tagit flest medaljer

# En meddalj per lag, för att få korrekt OS statistik 
ita_medals_unique = (
    ita[ita["Medal"] != "None"]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)

medals_by_sport = (
    ita_medals_unique
    .groupby("Sport")["Medal"]
    .count()
    .sort_values(ascending=False)
)

fig = px.bar(
    medals_by_sport.head(10),
    title="Top 10 sporter där Italien tagit flest medaljer",
    labels={"index": "Sport", "value": "Antal medaljer"}
)
fig.show()

# Top 5 sporter med flest italienska deltagare

participants_by_sport = (
    ita
    .drop_duplicates(subset=["Games", "ID"])   # en deltagare per sport + OS
    .groupby("Sport")["ID"]
    .count()
    .sort_values(ascending=False)
)

fig = px.bar(
    participants_by_sport.head(5),
    title="Top 5 sporter med flest italienska deltagare",
    labels={"index": "Sport", "value": "Antal deltagare"}
)
fig.show()



In [ ]:
# Sommar-OS
ita_summer_unique = (
    ita[
        (ita["Medal"] != "None") &
        (ita["Season"] == "Summer")
    ]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)

medals_by_games_summer = (
    ita_summer_unique
    .groupby("Games")["Medal"]
    .count()
    .reset_index()
)

fig = px.line(
    medals_by_games_summer,
    x="Games",
    y="Medal",
    title="Medaljer per OS - Italien Sommar",
    markers=True
)
fig.show()

# Vinter-OS
ita_winter_unique = (
    ita[
        (ita["Medal"] != "None") &
        (ita["Season"] == "Winter")
    ]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)

medals_by_games_winter = (
    ita_winter_unique
    .groupby("Games")["Medal"]
    .count()
    .reset_index()
)

fig = px.line(
    medals_by_games_winter,
    x="Games",
    y="Medal",
    title="Medaljer per OS - Italien Vinter",
    markers=True
)
fig.show()


In [ ]:
# Histogram över åldrar för italienska 

fig = px.histogram(
    ita,
    x="Age",
    nbins=30,                     
    title="Åldersfördelning - Italienska OS-idrottare",
    labels={"Age": "Ålder"},
)

fig.show()


In [ ]:
fig = px.pie(ita, names="Sex", title="Könsfördelning – Italien")
fig.show()

In [ ]:
# Antal deltagare per OS – Sommar 
ita_summer_participants = (
    ita[ita["Season"] == "Summer"]
    .drop_duplicates(subset=["Games", "ID"])     # en deltagare per OS
    .groupby("Games")["ID"]
    .count()
    .reset_index()
)

fig = px.line(
    ita_summer_participants,
    x="Games",
    y="ID",
    title="Antal italienska deltagare per OS - Sommar",
    markers=True,
    labels={"ID": "Antal deltagare"}
)
fig.show()


ita_winter_participants = (
    ita[ita["Season"] == "Winter"]
    .drop_duplicates(subset=["Games", "ID"])     # en deltagare per OS
    .groupby("Games")["ID"]
    .count()
    .reset_index()
)

fig = px.line(
    ita_winter_participants,
    x="Games",
    y="ID",
    title="Antal italienska deltagare per OS - Vinter",
    markers=True,
    labels={"ID": "Antal deltagare"}
)
fig.show()



# Fencing

In [ ]:
fencing = ita[ita["Sport"] == "Fencing"].copy()

fencing_unique_medals = (
    fencing[fencing["Medal"] != "None"]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)


In [ ]:
medals_by_type = (
    fencing_unique_medals
    .pivot_table(
        index="Year",
        columns="Medal",
        values="ID",      # bara för count
        aggfunc="count",
        fill_value=0
    )
    .reset_index()
)

fig1 = px.bar(
    medals_by_type,
    x="Year",
    y=["Gold", "Silver", "Bronze"],
    title="Italy Fencing - Medals per Year",
    labels={"value": "Number of Medals", "variable": "Medal Type"}
)
fig1.update_layout(barmode="stack")
fig1.show()

# Beräkna totala medaljer per år
medals_type_cols = [c for c in medals_by_type.columns if c in ["Gold", "Silver", "Bronze"]]
medals_by_type["Total"] = medals_by_type[medals_type_cols].sum(axis=1)

fig2 = px.line(
    medals_by_type,
    x="Year",
    y="Total",
    markers=True,
    title="Italy Fencing - Total Medals per Year",
    labels={"Total": "Total Medals"}
)
fig2.show()


In [ ]:
# Fäktning för alla länder
fencing_all = df[df["Sport"] == "Fencing"].copy()

fencing_all_unique_medals = (
    fencing_all[fencing_all["Medal"] != "None"]
    .drop_duplicates(subset=["Games", "Event", "Medal"])
)

medals_country = (
    fencing_all_unique_medals
    .groupby("NOC")["Medal"]
    .count()
    .reset_index()
    .sort_values("Medal", ascending=False)
)

px.bar(
    medals_country.head(20),
    x="NOC",
    y="Medal",
    title="Fencing  Medal Distribution by Country",
    labels={"NOC": "Country", "Medal": "Number of Medals"}
).show()


In [ ]:
fig = px.histogram(
    fencing,
    x="Age",
    nbins=30,
    marginal="box",
    title="Age Distribution in Fencing"
)
fig.show()


In [ ]:
medals_year = (
    fencing_unique_medals
    .groupby("Year")["Medal"]
    .count()
    .reset_index()
)

px.line(
    medals_year,
    x="Year",
    y="Medal",
    markers=True,
    title="Fencing Italy  Medal Count Over Time ",
    labels={"Medal": "Number of Medals"}
).show()


In [ ]:
fencing["Medalist"] = fencing["Medal"].notna()

fig = px.histogram(
    fencing,
    x="Age",
    color="Medalist",
    barmode="overlay",
    nbins=30,
    title="Age Distribution - Medalists vs Non-Medalists"
)
fig.show()


In [ ]:
# Medaljer per eent
medals_by_event = (
    fencing_unique_medals
    .groupby("Event")["Medal"]
    .count()
    .sort_values(ascending=False)  
    .reset_index()
)

fig = px.bar(
    medals_by_event,
    x="Event",
    y="Medal",
    title="Italy Fencing - Medals per Event",
    labels={"Medal": "Number of Medals", "Event": "Event"}
)

fig.update_xaxes(tickangle=45)
fig.show()
